# Ordinary Differential Equations
## Euler's method
An ODE is solved numerically by starting with an initial
value of the dependent variable $y(t = 0) ≡ y_0$, and using the derivative function $f(t, y)$
to advance $y_0$ one small step h forward in time to $y(t = h) ≡ y_1$. The algorithm then
just keeps repeating itself, treating the new values for $y$ as new initial conditions for
the next step. Euler’s rule does this via a straightforward application of the forward-difference algorithm for the derivative:
$$y_{n+1} \approx y_n + hf(t_n,y_n)$$
here $y_n ≡ y(t_n)$ is the value of y at time $t_n$. Aside from its use in initiating other
algorithms, Euler’s method is not accurate enough for scientific work. The error here goes as $h^2$ (can be seen from taylor series expansion). 

## Runge-Kutta Methods
The most popular and stable general technique of solving ODE’s is a set of methods known as “Runge-Kutta” methods. Of these family of methods the fourth-order Runge-Kutta method (rk4) is proven to be robust and capable of industrial strength work. The algorithm is based upon the formal integral of the
differential equation:
$\frac{dy}{dt}=f(t,y) \Rightarrow y(t) = \int f(t,y)dy \Rightarrow y_{n+1} = y_n + \int_{t_n}^{t_{n+1}} f(t,y) dt$. The critical idea here is to expand the integrand around the midpoint of the integration interval and thereby obtain fourth-order error precision via the cancellation of the $h$ and $h^3$ terms. 

Let's look at the mechanism of second-order Runge-Kutta method. For this, we define y as the function we’re looking for, and g as the derivative of y. Now
g is in general a function of both y and t, so the second derivative of y is,
using the chain rule
$$y'' = \frac{d}{dt}y' = \frac{d}{dt}g(y,t) = \frac{\partial g}{\partial t} + \frac{\partial g}{\partial y} \frac{dy}{dt}$$

where $g_k = \frac{\partial g}{\partial t}$

Similarily $y''' = g_{tt} + 2gg_{ty} + g^2 g_{yy} + g g_y^2 + g_t g_y$

Now, The Taylor expansion of $y(t + ∆t)$ is
$$y(t + ∆t) = y(t)
+ g∆t
+
\frac{∆t^2}{2!} (g_t + g_yg)
+
\frac{∆t^3}{3!} (g_{tt} + 2gg_{ty} + g^2 g_{yy} + g g_y^2 + g_t g_y)
+ O(∆t^4)$$

where O(∆t^4) is fourth order terms and higher.

Now, the main idea is to write $y(t)$ as a linear combination of some set of polynomials:
$$y(t + ∆t) = y(t) + α_1k_1 + α_2k_2 + · · · + α_nk_n$$
such that $k_1 = ∆tg(y, t) \\
k_2 = ∆tg(y + ν_{21}k_1, t + ν_{21}∆t) \\
k_3 = ∆tg(y + ν_{31}k_1 + ν_{32}k_2, t + ν_{31}∆t + ν_{32}∆t) \\
. \\ 
. \\ 
. \\
k_n = ∆tg(y +\sum_{l=1}^{n-1}v_{nl}k_l, t +∆t\sum_{l=1}^{n-1}v_{nl}$

where α and ν are parameters to be determined.

Now, we compare this with the Taylor series expansion we obtained to determine the coefficients.
$$y(t + ∆t) = y + α_1k_1 + α_2k_2 \\
= y + α_1[∆tg(y, t)] + α_2[∆tg(y + ν_{21}k_1), t + ν_{21}∆t]$$

The first and second terms are exact already, so we expand only the
third term.
$$k_2 = ∆t[g + ν_{21}k_1g_y + ν_{21}∆tg_t + O(∆t^
2
)]
= ∆tg + ν_{21}∆t^
2
gg_y + ν_{21}∆t^
2
g_t + O(∆t^
3
)$$

Substituting this $k_2$ we get
$$y(t + ∆t) = y + [(α_1 + α_2)g]∆t + [α_2ν_{21}(gg_y + g_t)]∆t^
2$$
Comparing this with the taylor series expansion equation, we get the the following equations for our parameters.
$$α_1 + α_2 = 1\\
α_2ν_{21} =\frac{1}{2}$$
There are three unknowns here, and only two equations, so the system
is underspecified. We can pick one value, then. If we choose ν_{21} = 1, then
α_1 and α_2 are each $\frac{1}{2}$. These values give us a second-order Runge-Kutta method:


$$y(t + ∆t) = y +\frac{1}{2}
k_1 +
\frac{1}{2}
k_2 + O(∆t^3
)$$

Conceptually, this second-order Runge-Kutta method is equivalent to
taking the average of the slope at t and at t + ∆t and using that average
slope in Euler’s method to find the new value $y(t + ∆t)$. Here we give the code for rk2 method:

In [2]:
def rk2a( f, x0, t ):
    """Second-order Runge-Kutta method to solve x' = f(x,t) with x(t[0]) = x0.

    USAGE:
        x = rk2a(f, x0, t)

    INPUT:
        f     - function of x and t equal to dx/dt.  x may be multivalued,
                in which case it should a list or a NumPy array.  In this
                case f must return a NumPy array with the same dimension
                as x.
        x0    - the initial condition(s).  Specifies the value of x when
                t = t[0].  Can be either a scalar or a list or NumPy array
                if a system of equations is being solved.
        t     - list or NumPy array of t values to compute solution at.
                t[0] is the the initial condition point, and the difference
                h=t[i+1]-t[i] determines the step size h.

    OUTPUT:
        x     - NumPy array containing solution values corresponding to each
                entry in t array.  If a system is being solved, x will be
                an array of arrays.

    NOTES:
        This version is based on the algorithm presented in "Numerical
        Analysis", 6th Edition, by Burden and Faires, Brooks-Cole, 1997.
    """

    n = len( t )
    x = numpy.array( [ x0 ] * n )
    for i in xrange( n - 1 ):
        h = t[i+1] - t[i]
        k1 = h * f( x[i], t[i] ) / 2.0
        x[i+1] = x[i] + h * f( x[i] + k1, t[i] + h / 2.0 )

    return x

As you can guess from what we said before, the taylor series can be expanded to obtain higher-order Runge-Kutta methods.The fourth-order Runge Kutta method formula is as follows.
$$y (t + ∆t) = y (t) + \frac{1}{6}
(k_1 + 2k_2 + 2k_3 + k_4) + O(∆t^4)
$$